# Prelude

In [ ]:
%matplotlib notebook
import matplotlib.pyplot as plt
import numpy as np

from bgcellmodels.common import analysis
from bgcellmodels.mechanisms import noise, synapses

import bgcellmodels.models.STN.GilliesWillshaw.gillies_model as gillies_model

import neuron
h = neuron.h

# Create cell

In [ ]:
soma, dends, stims = gillies_model.stn_cell_gillies()

# Noise

In [ ]:
sim_dur = 200.0
sim_dt = 0.025

# std of 0.005 - 0.01 seems to be good value
mean, std = 0.0, 0.0075 # [mA/cm2]

h.cvode_active(0)
h.cvode.active(0)

## InGauss Method

In [ ]:
# RNG for noise source
# rng = h.Random()
# rng.MCellRan4(1, 1e9) # high_index can also be set using rng.seq()
# rng.normal(0, 1)

# gstim = h.InGauss(soma(0.5))
# gstim.mean = mean
# gstim.stdev = std * 1e-2 * sum((seg.area() for seg in soma))
# gstim.delay = 5
# gstim.per = sim_dt
# gstim.dur = 1e9
# gstim.noiseFromRandom(rng)

# CONCLUSION
# - increases simulation time by factor 30 (with or without RNG attached)
# - 500 ms simulation = 1160 ms walltime

## Ingauss2 Method

In [ ]:
rng = h.Random()
rng.MCellRan4(1, 1e9) # high_index can also be set using rng.seq()
rng.normal(0, 1)

gstim = h.ingauss2(soma(0.5))
noise_amp = std # [mA/cm2]
gstim.mean = mean
gstim.stdev = noise_amp * 1e-2 * sum((seg.area() for seg in soma))
gstim.noiseFromRandom(rng)

# CONCLUSION
# - 500 ms simulation = 1180 ms walltime

## Vector.play() Method

In [ ]:
# noise_samples = np.random.normal(mean, std, int(sim_dur/sim_dt))
# noise_avec = h.Vector(noise_samples)
# noise_tvec = h.Vector(np.arange(0, sim_dur+sim_dt, sim_dt))

# stim = h.IClamp(soma(0.5))
# stim.dur = 1e9
# noise_avec.play(stim, stim._ref_amp, noise_tvec)

# CONCLUSION
# - increases simulation time by factor 20
# - 500 ms simulation = 1200 ms walltime

# Simulation

In [ ]:
# Define traces
rec_secs = {
    'soma': soma,
    'gstim': gstim,
}

trace_specs = {
    'V_soma': {'var':'v', 'sec':'soma', 'loc':0.5},
    'NaF_soma': {'var':'ina', 'mech': 'Na', 'sec':'soma', 'loc':0.5},
    'I_noise': {'pointp':'gstim', 'var':'i'},
}

# Record
rec_dt = 0.05
vec_dict, markers = analysis.recordTraces(rec_secs, trace_specs, rec_dt)

In [ ]:
%%time

# Init and run simulation
h.dt = sim_dt
h.celsius = 35.0
h.set_aCSF(4) # Hoc function defined in Gillies code
h.v_init = -68.0
h.tstop = sim_dur
h.init()
h.run()
# nrnsim.run(h.tstop, h.dt)

In [ ]:
# Automatic plots, no customization
figs_vm = analysis.plotTraces(vec_dict, rec_dt, traceSharex=True) # yRange=(-80,40),

In [ ]:
# Plot recorded traces
plt.figure()
v_soma = vec_dict['V_soma'].as_numpy()
t_soma = np.arange(len(v_soma)) * rec_dt
plt.plot(t_soma, v_soma)
plt.grid(True)

# Plot synaptic traces
fig, ax = plt.subplots()
ax.plot(t_soma, vec_dict['I_noise'].as_numpy(), 'b', label='I_noise')
ax.set_ylabel('current (nA)')
ax.legend()